In [1]:
path1  = 'C:/PycharmProjects/yjz/Extension for hts/JayCode/Models/'
from Invtsim import * 
import numpy as np
import pandas as pd
from scipy.stats import norm
import warnings
warnings.simplefilter("ignore")
from tqdm import tqdm

In [2]:
fcst    = pd.read_pickle(f"{path1}fcsts_base2cohe_tdfp.pkl")
fitt    = pd.read_pickle(f"{path1}fitts_base2cohe_tdfp.pkl")

ets     = fcst.iloc[:,4:]
lgb     = fcst.iloc[:,:4]

etsf    = fitt.iloc[:,4:]
lgbf    = fitt.iloc[:,:4]

test    = pd.read_pickle(f"{path1}future_28.pkl").reset_index(drop=True)
truth   = test["y"]
history = pd.read_pickle(f"{path1}thepast_1913.pkl")

ets_resid = pd.concat({ f"{etsf.columns[i]}": history - etsf.iloc[:,i] for i in range(4)},axis=1)
lgb_resid = pd.concat({ f"{lgbf.columns[i]}": history - lgbf.iloc[:,i] for i in range(4)},axis=1)

## Verify:

In [3]:
lp = {"l1" :(0,1),
      "l2" :(1,(1+3)),
      "l4" :((1+3),(1+3+3)),
      "l3" :((1+3+3),(1+3+3+10)),
      "l5" :((1+3+10+3),(1+3+10+3+7)),
      "l6" :((1+3+10+3+7),(1+3+10+3+7+9)),
      "l7" :((1+3+10+3+7+9),(1+3+10+3+7+9+21)),
      "l8" :((1+3+10+3+7+9+21),(1+3+10+3+7+9+21+30)),
      "l10" :((1+3+10+3+7+9+21+30),(1+3+10+3+7+9+21+30+3049)),
      "l9":((1+3+10+3+7+9+21+30+3049),(1+3+10+3+7+9+21+30+3049+70)),
      "l11":((1+3+10+3+7+9+21+30+70+3049),(1+3+10+3+7+9+21+30+70+3049+9147)),
      "l12":((1+3+10+3+7+9+21+30+70+3049+9147),(1+3+10+3+7+9+21+30+70+3049+9147+30490))}

def check(lp:dict, df:pd.DataFrame, col:str = 'backlog', gap:int = 28):
    asl = []
    for i in lp.keys():
        bt = df[lp[i][0]*gap:lp[i][1]*gap]
        l1 = bt[col]
        if col == 'backlog':
            print(f"{len(l1[l1==0])/len(l1)}")
            asl.append(len(l1[l1==0])/len(l1))
        else:
            asl.append(l1.mean())
    return asl

In [4]:
def loop(fcst, truth, residual, NAME:str,
         gap1: int = 28, gap2:int = 1913,
         n = None, 
         df_list:list = [], 
         L_ = 3):
    if n == None:
        n = int(len(fcst)/gap1)
    for i in tqdm(range(n)):
        fcst_1  = fcst[i*gap1:(i+1)*gap1]
        truth_1 = truth[i*gap1:(i+1)*gap1]
        res_1   = residual[i*gap2:(i+1)*gap2]
        df = InvtSim(fcst = fcst_1, truth = truth_1, residual = res_1, name = NAME, L=L_)
        res = df.ob_all_t()
        df_list.append(res.reset_index(drop = True))
    #results = pd.concat(res)
    print("-----------DOWN!------------")
    return pd.concat(df_list).reset_index(drop=True)

In [5]:
df = loop(fcst = fcst['lgb_base'], truth = truth, residual = lgb_resid['lgb_base'], NAME = 'lgb_base', df_list = [],L_=1,n=1)
a = df[['cb_90','cb_95','cb_99']]#[['ot_90','ip_90','ot_95','ip_95','ot_99','ip_99']]#[['backlog_90','backlog_95','backlog_99']]#

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 202.09it/s]

-----------DOWN!------------


In [6]:
df[['cb_90','cb_95','cb_99']]#[['ot_90','ip_90','ot_95','ip_95','ot_99','ip_99','cb_90','cb_95','cb_99']]
#[['sst_99','sst_90','sst_95']]
#[['ot_90','ip_90','ot_95','ip_95','ot_99','ip_99','cb_90','cb_95','cb_99']]#[['sst_99','ot_99']]#[['backlog_90','backlog_95','backlog_99']]#

,cb_90,cb_95,cb_99
0,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000
4,30172.662835,60766.706862,287977.356283
5,3348.464339,4137.843370,0.000000
6,13986.758784,26596.464978,109932.411726
7,18260.857708,35619.562705,156947.499883
8,12866.321707,24231.097814,97607.603875
9,15780.565984,30383.391288,129664.290920


In [9]:
50624.80952/99

511.3617123232323

## LEAD TIME 1:

In [14]:
Invt_df = []
nam_lgb = [ "lgb_base", "lgb_bu", "lgb_td", "lgb_mint"]
nam_ets = [ "ets_base", "ets_bu", "ets_td", "ets_mint"]
lead_time = 1
for i in nam_lgb:
    df = loop(fcst = fcst[i], truth = truth, residual = lgb_resid[i], NAME = i, df_list = [],L_=lead_time)# L_=3
    Invt_df.append(df)
pd.concat(Invt_df).to_pickle(f"lgbInvtSim_L{lead_time}.pkl")

Invt_df = []
for i in nam_ets:
    df = loop(fcst = fcst[i], truth = truth, residual = ets_resid[i], NAME = i, df_list = [],L_=lead_time)
    Invt_df.append(df)
pd.concat(Invt_df).to_pickle(f"etsInvtSim_L{lead_time}.pkl")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:41<00:00, 422.94it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:41<00:00, 423.29it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:42<00:00, 416.61it/s]


...

-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:39<00:00, 432.47it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:39<00:00, 432.46it/s]


-----------DOWN!------------


## LEAD TIME 3:

In [7]:
Invt_df = []
nam_lgb = [ "lgb_base", "lgb_bu", "lgb_td", "lgb_mint"]
nam_ets = [ "ets_base", "ets_bu", "ets_td", "ets_mint"]
lead_time = 3
for i in nam_lgb:
    df = loop(fcst = fcst[i], truth = truth, residual = lgb_resid[i], NAME = i, df_list = [],L_=lead_time)# L_=3
    Invt_df.append(df)
pd.concat(Invt_df).to_pickle(f"lgbInvtSim_L{lead_time}.pkl")

Invt_df = []
for i in nam_ets:
    df = loop(fcst = fcst[i], truth = truth, residual = ets_resid[i], NAME = i, df_list = [],L_=lead_time)
    Invt_df.append(df)
pd.concat(Invt_df).to_pickle(f"etsInvtSim_L{lead_time}.pkl")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:48<00:00, 395.27it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [04:54<00:00, 145.69it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [03:17<00:00, 216.47it/s]


...

-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:39<00:00, 431.52it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:39<00:00, 432.39it/s]


-----------DOWN!------------


## LEAD TIME 7:

In [8]:
Invt_df = []
nam_lgb = [ "lgb_base", "lgb_bu", "lgb_td", "lgb_mint"]
nam_ets = [ "ets_base", "ets_bu", "ets_td", "ets_mint"]
lead_time = 7
for i in nam_lgb:
    df = loop(fcst = fcst[i], truth = truth, residual = lgb_resid[i], NAME = i, df_list = [],L_=lead_time)# L_=3
    Invt_df.append(df)
pd.concat(Invt_df).to_pickle(f"lgbInvtSim_L{lead_time}.pkl")

Invt_df = []
for i in nam_ets:
    df = loop(fcst = fcst[i], truth = truth, residual = ets_resid[i], NAME = i, df_list = [],L_=lead_time)
    Invt_df.append(df)
pd.concat(Invt_df).to_pickle(f"etsInvtSim_L{lead_time}.pkl")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:48<00:00, 394.25it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:48<00:00, 393.31it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:49<00:00, 392.89it/s]


...

-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:47<00:00, 400.35it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:46<00:00, 400.69it/s]


-----------DOWN!------------


## LEAD TIME 14:

In [9]:
Invt_df = []
nam_lgb = [ "lgb_base", "lgb_bu", "lgb_td", "lgb_mint"]
nam_ets = [ "ets_base", "ets_bu", "ets_td", "ets_mint"]
lead_time = 14
for i in nam_lgb:
    df = loop(fcst = fcst[i], truth = truth, residual = lgb_resid[i], NAME = i, df_list = [],L_=lead_time)# L_=3
    Invt_df.append(df)
pd.concat(Invt_df).to_pickle(f"lgbInvtSim_L{lead_time}.pkl")

Invt_df = []
for i in nam_ets:
    df = loop(fcst = fcst[i], truth = truth, residual = ets_resid[i], NAME = i, df_list = [],L_=lead_time)
    Invt_df.append(df)
pd.concat(Invt_df).to_pickle(f"etsInvtSim_L{lead_time}.pkl")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:59<00:00, 358.85it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:58<00:00, 360.82it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:59<00:00, 357.53it/s]


...

-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:58<00:00, 360.87it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:57<00:00, 363.58it/s]


-----------DOWN!------------


## Inventory Reconciliation

In [8]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.simplefilter("ignore")
lgbsim = pd.read_pickle("lgbInvtSim_L3.pkl")
etssim = pd.read_pickle("etsInvtSim_L3.pkl")

In [9]:
lgbsim

,name,true_demand,forecasts,ot_90,sst_90,ip_90,net_90,backlog_90,ch_90,cb_90,...,backlog_95,ch_95,cb_95,ot_99,sst_99,ip_99,net_99,backlog_99,ch_99,cb_99
0,lgb_base,38793,37936.346099,0.000000,942.565094,69932.000000,69932.000000,0.000000,69932.000000,0.00000,...,0.000000,69932.000000,0.000000,236.040956,1710.999668,70168.040956,69932.000000,0.000000,69932.000000,0.000000
1,lgb_base,35487,35346.413219,34610.738143,942.565094,69055.738143,34445.000000,0.000000,34445.000000,0.00000,...,0.000000,34445.000000,0.000000,35143.131762,1710.999668,69824.172717,34681.040956,0.000000,34681.040956,0.000000
2,lgb_base,34445,33967.281969,39139.878477,942.565094,73750.616620,34610.738143,0.000000,34610.738143,0.00000,...,0.000000,34877.942250,0.000000,39139.878477,1710.999668,74519.051194,35379.172717,0.000000,35379.172717,0.000000
3,lgb_base,34732,34286.477860,49990.476227,942.565094,89009.092847,-121.261857,121.261857,0.000000,1091.35671,...,0.000000,145.942250,0.000000,49990.476227,1710.999668,89777.527421,883.213673,0.000000,883.213673,0.000000
4,lgb_base,42896,38999.291696,51379.247619,942.565094,97492.340466,-8406.523713,8406.523713,0.000000,75658.71342,...,7872.115499,0.000000,149570.194482,51379.247619,1710.999668,98260.775040,-6869.654566,6869.654566,0.000000,680095.801985
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199515,lgb_mint,0,0.096519,0.000000,0.866392,1.398920,2.349361,0.000000,2.349361,0.00000,...,0.000000,2.840581,0.000000,0.000000,1.572726,2.105254,3.762029,0.000000,3.762029,0.000000
1199516,lgb_mint,0,0.113224,0.000000,0.866392,1.398920,2.349361,0.000000,2.349361,0.00000,...,0.000000,2.840581,0.000000,0.000000,1.572726,2.105254,3.762029,0.000000,3.762029,0.000000
1199517,lgb_mint,0,0.089249,0.000000,0.866392,1.398920,2.349361,0.000000,2.349361,0.00000,...,0.000000,2.840581,0.000000,0.000000,1.572726,2.105254,3.762029,0.000000,3.762029,0.000000
1199518,lgb_mint,0,0.145623,0.000000,0.866392,1.398920,2.349361,0.000000,2.349361,0.00000,...,0.000000,2.840581,0.000000,0.000000,1.572726,2.105254,3.762029,0.000000,3.762029,0.000000


In [8]:
lgbsim = pd.read_pickle("lgbInvtSim.pkl")

In [10]:
lgbsim

,name,true_demand,forecasts,ot_90,ip_90,net_90,backlog_90,ch_90,cb_90,ot_95,...,net_95,backlog_95,ch_95,cb_95,ot_99,ip_99,net_99,backlog_99,ch_99,cb_99
0,lgb_base,38793,37936.346099,0.000000,70954.109013,70954.109013,0.000000,70954.109013,0.000000,0.000000,...,71243.862716,0.000000,71243.862716,0.000000,0.000000,71787.392474,71787.392474,0.000000,71787.392474,0.000000
1,lgb_base,35487,35346.413219,33676.098158,35467.109013,35467.109013,0.000000,35467.109013,0.000000,33678.344816,...,35756.862716,0.000000,35756.862716,0.000000,33682.559173,36300.392474,36300.392474,0.000000,36300.392474,0.000000
2,lgb_base,34445,33967.281969,72824.764518,34698.207171,1022.109013,0.000000,1022.109013,0.000000,72829.502419,...,1311.862716,0.000000,1311.862716,0.000000,72838.389933,35537.951648,1855.392474,0.000000,1855.392474,0.000000
3,lgb_base,34732,34286.477860,89144.057315,72790.971689,-33709.890987,33709.890987,0.000000,640487.928748,89147.941815,...,-33420.137284,33420.137284,0.000000,634982.608395,89155.228490,73644.341581,-32876.607526,32876.607526,0.000000,624655.542990
4,lgb_base,42896,38999.291696,67695.864735,119039.029003,-42929.792829,42929.792829,0.000000,815666.063750,67694.252816,...,-42637.792468,42637.792468,0.000000,810118.056889,67691.229123,119903.570071,-42090.048352,42090.048352,0.000000,799710.918696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199515,lgb_mint,0,0.096519,0.000000,0.931117,0.931117,0.000000,0.931117,0.000000,0.000000,...,1.038955,0.000000,1.038955,0.000000,0.000000,1.241242,1.241242,0.000000,1.241242,0.000000
1199516,lgb_mint,0,0.113224,0.000000,0.931117,0.931117,0.000000,0.931117,0.000000,0.000000,...,1.038955,0.000000,1.038955,0.000000,0.000000,1.241242,1.241242,0.000000,1.241242,0.000000
1199517,lgb_mint,0,0.089249,0.000000,0.931117,0.931117,0.000000,0.931117,0.000000,0.000000,...,1.038955,0.000000,1.038955,0.000000,0.000000,1.241242,1.241242,0.000000,1.241242,0.000000
1199518,lgb_mint,0,0.145623,0.000000,0.931117,0.931117,0.000000,0.931117,0.000000,0.000000,...,1.038955,0.000000,1.038955,0.000000,0.000000,1.241242,1.241242,0.000000,1.241242,0.000000
